In [ ]:
import sys
sys.path.append('../src/')
import os
import re
import nibabel as nib
import numpy as np
import scipy
import math

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
%load_ext autoreload
%autoreload 2

from finetune_3D import *
from logistic_regression import *
from models import *
from utils import *

In [ ]:
df = load_experiment('/cluster/home/eharve06/extrapolating-classifier-accuracy-to-bigger-datasets/experiments/WMD/n=414_random_state=3001.csv')
print(open('/cluster/home/eharve06/extrapolating-classifier-accuracy-to-bigger-datasets/experiments/WMD/n=414_random_state=3001.txt').read())
df.tail()

In [ ]:
plt.plot(df.epoch.to_list(), df.train_loss.to_list())
plt.plot(df.epoch.to_list(), df.val_loss.to_list())
plt.plot(df.epoch.to_list(), df.test_loss.to_list())

In [ ]:
plt.plot(df.epoch.to_list(), np.array(df.train_auroc.to_list())[:,0])
plt.plot(df.epoch.to_list(), np.array(df.val_auroc.to_list())[:,0])
plt.plot(df.epoch.to_list(), np.array(df.test_auroc.to_list())[:,0])

In [ ]:
val_performance = np.array(df.val_loss.to_list())
averaged_performance = np.array([sum(val_performance[index-30:index]) for index in range(30, len(val_performance))])
print(val_performance.shape)
print('best epoch: {}'.format(30+np.argmin(averaged_performance)))
print('val_auroc: {}'.format(df.val_auroc.to_list()[30+np.argmin(averaged_performance)]))
print('test_auroc: {}'.format(df.test_auroc.to_list()[30+np.argmin(averaged_performance)]))

In [ ]:
columns = ['n', 'random_state', 'train_BA', 'train_auroc', 'val_BA', 'val_auroc', 'test_BA', 'test_auroc']
df = pd.DataFrame(columns=columns)

ns = [60, 94, 147, 230, 360, 414, 475, 546, 627]
ns = [60, 94, 147, 230, 360, 414, 475]
random_states = [1001, 2001, 3001, 4001, 5001, 6001, 7001, 8001, 9001, 10001, 11001, 12001, 13001, 14001, 15001]
experiments_path = '/cluster/home/eharve06/extrapolating-classifier-accuracy-to-bigger-datasets/experiments/WMD'

for model_index, (n, random_state) in enumerate(itertools.product(ns, random_states)):    
    temp_df = load_experiment('{}/n={}_random_state={}.csv'\
                         .format(experiments_path, n, random_state))
    val_performance = np.array(temp_df.val_auroc.to_list())
    averaged_performance = np.array([sum(val_performance[index-30:index]) for index in range(30, len(val_performance))])
    #averaged_performance[np.all(np.array(temp_df.train_auroc.to_list())[30:] < val_performance[30:], axis=-1)] = 0
    train_loss, train_BA, train_auroc, val_loss, val_BA, val_auroc, test_loss, test_BA, test_auroc = temp_df.iloc[30+np.argmax(averaged_performance)].to_list()[1:]
    row = [n, random_state, train_BA, train_auroc, val_BA, val_auroc, test_BA, test_auroc]
    df.loc[model_index] = row
    print('best epoch: {}'.format(30+np.argmax(averaged_performance)))
    print('val_loss: {:0.4f}'.format(temp_df.val_loss.to_list()[30+np.argmax(averaged_performance)]))
    print(open('{}/n={}_random_state={}.txt'.format(experiments_path, n, random_state)).read())

In [ ]:
df

In [ ]:
def get_alpha(n):
    if n <= 360:
        return 1.0
    else:
        return 0.3

def plot_experiment(df, labels, random_states=[1001, 2001, 3001]):
    df = df[df.random_state.isin(random_states)]
    colors = ['#2ca02c', '#ff7f0e', '#9467bd', '#1f77b4', '#d62728', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf', '#ff9896', '#aec7e8', '#ff0000', '#00ff00', '#0000ff']
    ncols = len(labels) if len(labels) < 4 else 4
    nrows = math.ceil(len(labels)/4)
    fig, axs = plt.subplots(ncols=ncols, nrows=nrows, figsize=(ncols*4, nrows*3))
    for label_index, _ in enumerate(labels):       
        for color, random_state in zip(colors, random_states):
            # Plot data
            temp_df = df[df.random_state==random_state]
            np.array(axs).flatten()[label_index]\
            .scatter(temp_df.n.to_numpy(),
                     np.array(temp_df.test_auroc.to_list())[:,label_index],
                     color = color,
                     alpha=list(map(get_alpha, df.n[df.random_state==random_state].to_list())),
                     label='seed {}'.format(random_state))
        np.array(axs).flatten()[label_index]\
        .set_xlim([50, 30000])
        np.array(axs).flatten()[label_index]\
        .set_ylim([0.5, 1.0])
        np.array(axs).flatten()[label_index]\
        .set_xscale('log')
        np.array(axs).flatten()[label_index]\
        .set_xlabel('Number of training samples')
        np.array(axs).flatten()[label_index]\
        .set_ylabel('{} AUROC'.format(labels[label_index]))
        np.array(axs).flatten()[label_index]\
        .legend()
        np.array(axs).flatten()[label_index]\
        .grid()
    fig.tight_layout()
    plt.show()
    
plot_experiment(df, ['WMD'], random_states)

In [ ]:
df = df.groupby('n').agg(lambda x: list(x))
df.test_auroc = df.test_auroc.apply(lambda x: np.array([np.mean(x)]))
df.random_state = df.random_state.apply(lambda x: 'mean')
df = df.reset_index()
plot_experiment(df, ['Alzheimer’s'], ['mean'])